In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
clean_data = Path("Resources/clean_data.csv")
clean_data_df = pd.read_csv(clean_data)

clean_data_df.head(10)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Data Scientist,239900,USD,239900,US,100,US,M
1,2023,SE,FT,Data Scientist,159900,USD,159900,US,100,US,M
2,2023,EN,FT,Data Analyst,162240,USD,162240,US,0,US,M
3,2023,EN,FT,Data Analyst,47840,USD,47840,US,0,US,M
4,2023,SE,FT,Data Engineer,130410,USD,130410,US,0,US,M
5,2023,SE,FT,Data Engineer,92000,USD,92000,US,0,US,M
6,2023,SE,FT,Data Science,270250,USD,270250,US,0,US,M
7,2023,SE,FT,Data Science,144000,USD,144000,US,0,US,M
8,2023,SE,FT,Business Intelligence Analyst,176875,USD,176875,US,0,US,M
9,2023,SE,FT,Business Intelligence Analyst,84750,USD,84750,US,0,US,M


In [3]:
no_remote_work_avg_sal = clean_data_df.loc[clean_data_df['remote_ratio'] == 0, 'salary_in_usd'].mean()
partial_remote_work_avg_sal = clean_data_df.loc[clean_data_df['remote_ratio'] == 50, 'salary_in_usd'].mean()
fully_remote_work_avg_sal = clean_data_df.loc[clean_data_df['remote_ratio'] == 100, 'salary_in_usd'].mean()

print(f"The average salary for data science professionals working less than 20% remote is ${round(no_remote_work_avg_sal)}.")
print(f"The average salary for data science professionals working 50% remote is ${round(partial_remote_work_avg_sal)}.")
print(f"The average salary for data science professionals working more than 80% remote is ${round(fully_remote_work_avg_sal)}.")

The average salary for data science professionals working less than 20% remote is $162559.
The average salary for data science professionals working 50% remote is $118427.
The average salary for data science professionals working more than 80% remote is $155694.


In [4]:
job_titles = set(clean_data_df['job_title'])
job_titles

{'AI Architect',
 'AI Developer',
 'AI Engineer',
 'AI Scientist',
 'AWS Data Architect',
 'Analytics Engineer',
 'Applied Machine Learning Engineer',
 'Applied Machine Learning Scientist',
 'Applied Scientist',
 'BI Analyst',
 'BI Data Engineer',
 'BI Developer',
 'Business Data Analyst',
 'Business Intelligence Analyst',
 'Business Intelligence Data Analyst',
 'Business Intelligence Developer',
 'Business Intelligence Engineer',
 'Business Intelligence Manager',
 'Business Intelligence Specialist',
 'Cloud Database Engineer',
 'Compliance Data Analyst',
 'Computer Vision Engineer',
 'Data Analyst',
 'Data Analytics Lead',
 'Data Analytics Manager',
 'Data Analytics Specialist',
 'Data Architect',
 'Data Developer',
 'Data Engineer',
 'Data Infrastructure Engineer',
 'Data Integration Engineer',
 'Data Integration Specialist',
 'Data Lead',
 'Data Management Analyst',
 'Data Management Specialist',
 'Data Manager',
 'Data Modeler',
 'Data Operations Analyst',
 'Data Operations Enginee

In [11]:
job_titles_count = clean_data_df['job_title'].nunique()
job_titles_count

85

In [13]:
grouped_data = clean_data_df.groupby('remote_ratio')['job_title'].unique()
grouped_data

remote_ratio
0      [Data Analyst, Data Engineer, Data Science, Bu...
50     [Business Data Analyst, Marketing Data Scienti...
100    [Data Scientist, Data Engineer, Machine Learni...
Name: job_title, dtype: object

In [21]:
new_df = clean_data_df[['remote_ratio', 'job_title', 'salary']].copy()
new_df

,remote_ratio,job_title,salary
0,100,Data Scientist,239900
1,100,Data Scientist,159900
2,0,Data Analyst,162240
3,0,Data Analyst,47840
4,0,Data Engineer,130410
...,...,...,...
7356,0,Machine Learning Engineer,134500
7357,0,Data Scientist,130000
7358,0,Data Scientist,90000
7359,0,Data Engineer,160000


In [39]:
# Group by 'remote_ratio' and 'job_title', then count the occurrences
count_df = new_df.groupby(['remote_ratio', 'job_title']).size().reset_index(name='count_title_remote_work')

# Find the five job titles with the most counts for each remote ratio
top_titles_per_remote_ratio = count_df.groupby('remote_ratio').apply(lambda x: x.nlargest(5, 'count_title_remote_work')).reset_index(drop=True)

# Group by 'remote_ratio' and 'job_title', then calculate the mean salary
average_salary_df = new_df.groupby(['remote_ratio', 'job_title'])['salary'].mean().reset_index()

# Merge the DataFrames on 'remote_ratio' and 'job_title'
result_df = pd.merge(top_titles_per_remote_ratio, average_salary_df, on=['remote_ratio', 'job_title'], how='left')

# Pivot the DataFrame
result_pivoted_df = result_df.pivot_table(index='job_title', columns='remote_ratio', values='salary', aggfunc='mean', fill_value=0)

# Rename the columns
result_pivoted_df.columns = [f'{col}_remote_ratio' for col in result_pivoted_df.columns]

# Reset the index
result_pivoted_df = result_pivoted_df.reset_index()

# Round the average salary columns
result_pivoted_df[['0_remote_ratio', '50_remote_ratio', '100_remote_ratio']] = result_pivoted_df[['0_remote_ratio', '50_remote_ratio', '100_remote_ratio']].round()
result_pivoted_df

,job_title,0_remote_ratio,50_remote_ratio,100_remote_ratio
0,Business Data Analyst,0.0,99500,0.0
1,Data Analyst,113976.0,100000,112227.0
2,Data Architect,0.0,0,176954.0
3,Data Engineer,155531.0,90000,156317.0
4,Data Scientist,171337.0,110000,168605.0
5,Machine Learning Engineer,196193.0,177000,192086.0
6,Research Scientist,191260.0,0,0.0
